# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 6780
Number of scenes: 130
Average number of sentences in each scene: 15.946153846153846
Number of lines: 2203
Average number of words in each line: 11.019972764412165

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [7]:
import numpy as np
from collections import Counter
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
#     words = text.split()
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [10]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = {
        ".":"<period>",
        ",":"<comma>",
        '"':"<quotation_mark>",
        ";":"<semicolon>",
        "!":"<exclamation_mark>",
        "?":"<question_mark>",
        "(":"<left_parentheses>",
        ")":"<right_parentheses>",
        "--":"<dash>",
        "\n":"<return>"
    }
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


C:\Users\zander\Anaconda3\envs\tf1.1\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [39]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    X = tf.placeholder(dtype=tf.int32, name='input',shape=(None, None))
    Y = tf.placeholder(dtype=tf.int32, name='targets',shape=(None, None))
    lr = tf.placeholder(dtype=tf.float32, name='learning_rate')
    return X, Y, lr


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [34]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop])
    
    # Getting an initial state of all zeros
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [38]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1), dtype=tf.float32)
    embed = tf.nn.embedding_lookup(embedding, input_data)    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [32]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function

#     cell, initial_state = get_init_cell(32, 64)
#     inputs = inputs.as_type(tf.float32)
#     embed = get_embed(inputs, len(vocab_to_int)+1, 300)
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [31]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [28]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    words_per_batch = batch_size*seq_length
    n_batchs = len(int_text)//words_per_batch
    if type(int_text) != np.ndarray:
        int_text = np.array(int_text)
    
    int_text = int_text[:n_batchs*words_per_batch]
    
    int_text = int_text.reshape((batch_size, -1))
    
    batchs = np.zeros((n_batchs, 2, batch_size, seq_length))
    for i,n in enumerate(range(0, int_text.shape[1], seq_length)):
        x = int_text[:, n:n+seq_length]
        # The targets, shifted by one
        y_temp = int_text[:, n+1:n+seq_length+1]
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        batchs[i][0] = x
        batchs[i][1] = y
    
    return batchs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [54]:
# Number of Epochs
num_epochs = 24
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 1e-3
# Show stats for every n number of batches
show_every_n_batches = True

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [55]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [57]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/67   train_loss = 8.346
Epoch   0 Batch    1/67   train_loss = 8.341
Epoch   0 Batch    2/67   train_loss = 8.332
Epoch   0 Batch    3/67   train_loss = 8.324
Epoch   0 Batch    4/67   train_loss = 8.314
Epoch   0 Batch    5/67   train_loss = 8.305
Epoch   0 Batch    6/67   train_loss = 8.289
Epoch   0 Batch    7/67   train_loss = 8.277
Epoch   0 Batch    8/67   train_loss = 8.256
Epoch   0 Batch    9/67   train_loss = 8.233
Epoch   0 Batch   10/67   train_loss = 8.205
Epoch   0 Batch   11/67   train_loss = 8.158
Epoch   0 Batch   12/67   train_loss = 8.128
Epoch   0 Batch   13/67   train_loss = 8.082
Epoch   0 Batch   14/67   train_loss = 8.021
Epoch   0 Batch   15/67   train_loss = 7.972
Epoch   0 Batch   16/67   train_loss = 7.920
Epoch   0 Batch   17/67   train_loss = 7.830
Epoch   0 Batch   18/67   train_loss = 7.798
Epoch   0 Batch   19/67   train_loss = 7.737
Epoch   0 Batch   20/67   train_loss = 7.633
Epoch   0 Batch   21/67   train_loss = 7.532
Epoch   0 

Epoch   2 Batch   49/67   train_loss = 5.875
Epoch   2 Batch   50/67   train_loss = 5.776
Epoch   2 Batch   51/67   train_loss = 5.906
Epoch   2 Batch   52/67   train_loss = 5.997
Epoch   2 Batch   53/67   train_loss = 6.038
Epoch   2 Batch   54/67   train_loss = 6.005
Epoch   2 Batch   55/67   train_loss = 6.006
Epoch   2 Batch   56/67   train_loss = 6.123
Epoch   2 Batch   57/67   train_loss = 5.927
Epoch   2 Batch   58/67   train_loss = 6.031
Epoch   2 Batch   59/67   train_loss = 6.081
Epoch   2 Batch   60/67   train_loss = 5.863
Epoch   2 Batch   61/67   train_loss = 6.035
Epoch   2 Batch   62/67   train_loss = 6.149
Epoch   2 Batch   63/67   train_loss = 5.965
Epoch   2 Batch   64/67   train_loss = 5.997
Epoch   2 Batch   65/67   train_loss = 5.855
Epoch   2 Batch   66/67   train_loss = 5.590
Epoch   3 Batch    0/67   train_loss = 5.742
Epoch   3 Batch    1/67   train_loss = 5.912
Epoch   3 Batch    2/67   train_loss = 5.741
Epoch   3 Batch    3/67   train_loss = 6.008
Epoch   3 

Epoch   5 Batch   31/67   train_loss = 5.602
Epoch   5 Batch   32/67   train_loss = 5.522
Epoch   5 Batch   33/67   train_loss = 5.627
Epoch   5 Batch   34/67   train_loss = 5.653
Epoch   5 Batch   35/67   train_loss = 5.600
Epoch   5 Batch   36/67   train_loss = 5.762
Epoch   5 Batch   37/67   train_loss = 5.837
Epoch   5 Batch   38/67   train_loss = 5.868
Epoch   5 Batch   39/67   train_loss = 5.556
Epoch   5 Batch   40/67   train_loss = 5.690
Epoch   5 Batch   41/67   train_loss = 5.381
Epoch   5 Batch   42/67   train_loss = 5.784
Epoch   5 Batch   43/67   train_loss = 5.547
Epoch   5 Batch   44/67   train_loss = 5.728
Epoch   5 Batch   45/67   train_loss = 5.706
Epoch   5 Batch   46/67   train_loss = 5.619
Epoch   5 Batch   47/67   train_loss = 5.523
Epoch   5 Batch   48/67   train_loss = 5.609
Epoch   5 Batch   49/67   train_loss = 5.503
Epoch   5 Batch   50/67   train_loss = 5.443
Epoch   5 Batch   51/67   train_loss = 5.602
Epoch   5 Batch   52/67   train_loss = 5.689
Epoch   5 

Epoch   8 Batch   13/67   train_loss = 5.485
Epoch   8 Batch   14/67   train_loss = 5.320
Epoch   8 Batch   15/67   train_loss = 5.416
Epoch   8 Batch   16/67   train_loss = 5.538
Epoch   8 Batch   17/67   train_loss = 5.316
Epoch   8 Batch   18/67   train_loss = 5.459
Epoch   8 Batch   19/67   train_loss = 5.456
Epoch   8 Batch   20/67   train_loss = 5.454
Epoch   8 Batch   21/67   train_loss = 5.365
Epoch   8 Batch   22/67   train_loss = 5.370
Epoch   8 Batch   23/67   train_loss = 5.173
Epoch   8 Batch   24/67   train_loss = 5.163
Epoch   8 Batch   25/67   train_loss = 5.300
Epoch   8 Batch   26/67   train_loss = 5.253
Epoch   8 Batch   27/67   train_loss = 5.505
Epoch   8 Batch   28/67   train_loss = 5.351
Epoch   8 Batch   29/67   train_loss = 5.436
Epoch   8 Batch   30/67   train_loss = 5.453
Epoch   8 Batch   31/67   train_loss = 5.321
Epoch   8 Batch   32/67   train_loss = 5.254
Epoch   8 Batch   33/67   train_loss = 5.354
Epoch   8 Batch   34/67   train_loss = 5.412
Epoch   8 

Epoch  10 Batch   63/67   train_loss = 5.242
Epoch  10 Batch   64/67   train_loss = 5.200
Epoch  10 Batch   65/67   train_loss = 5.050
Epoch  10 Batch   66/67   train_loss = 4.991
Epoch  11 Batch    0/67   train_loss = 5.000
Epoch  11 Batch    1/67   train_loss = 5.165
Epoch  11 Batch    2/67   train_loss = 4.901
Epoch  11 Batch    3/67   train_loss = 5.315
Epoch  11 Batch    4/67   train_loss = 5.204
Epoch  11 Batch    5/67   train_loss = 5.287
Epoch  11 Batch    6/67   train_loss = 5.070
Epoch  11 Batch    7/67   train_loss = 5.022
Epoch  11 Batch    8/67   train_loss = 5.190
Epoch  11 Batch    9/67   train_loss = 4.895
Epoch  11 Batch   10/67   train_loss = 5.034
Epoch  11 Batch   11/67   train_loss = 5.161
Epoch  11 Batch   12/67   train_loss = 5.004
Epoch  11 Batch   13/67   train_loss = 5.308
Epoch  11 Batch   14/67   train_loss = 5.132
Epoch  11 Batch   15/67   train_loss = 5.261
Epoch  11 Batch   16/67   train_loss = 5.346
Epoch  11 Batch   17/67   train_loss = 5.134
Epoch  11 

Epoch  13 Batch   45/67   train_loss = 5.137
Epoch  13 Batch   46/67   train_loss = 5.066
Epoch  13 Batch   47/67   train_loss = 5.042
Epoch  13 Batch   48/67   train_loss = 5.060
Epoch  13 Batch   49/67   train_loss = 4.967
Epoch  13 Batch   50/67   train_loss = 4.967
Epoch  13 Batch   51/67   train_loss = 5.132
Epoch  13 Batch   52/67   train_loss = 5.227
Epoch  13 Batch   53/67   train_loss = 5.310
Epoch  13 Batch   54/67   train_loss = 5.163
Epoch  13 Batch   55/67   train_loss = 5.148
Epoch  13 Batch   56/67   train_loss = 5.260
Epoch  13 Batch   57/67   train_loss = 5.011
Epoch  13 Batch   58/67   train_loss = 5.132
Epoch  13 Batch   59/67   train_loss = 5.198
Epoch  13 Batch   60/67   train_loss = 4.988
Epoch  13 Batch   61/67   train_loss = 5.178
Epoch  13 Batch   62/67   train_loss = 5.258
Epoch  13 Batch   63/67   train_loss = 5.100
Epoch  13 Batch   64/67   train_loss = 5.065
Epoch  13 Batch   65/67   train_loss = 4.910
Epoch  13 Batch   66/67   train_loss = 4.884
Epoch  14 

Epoch  16 Batch   26/67   train_loss = 4.913
Epoch  16 Batch   27/67   train_loss = 5.131
Epoch  16 Batch   28/67   train_loss = 4.966
Epoch  16 Batch   29/67   train_loss = 5.072
Epoch  16 Batch   30/67   train_loss = 5.107
Epoch  16 Batch   31/67   train_loss = 4.931
Epoch  16 Batch   32/67   train_loss = 4.907
Epoch  16 Batch   33/67   train_loss = 5.007
Epoch  16 Batch   34/67   train_loss = 5.047
Epoch  16 Batch   35/67   train_loss = 4.888
Epoch  16 Batch   36/67   train_loss = 5.092
Epoch  16 Batch   37/67   train_loss = 5.229
Epoch  16 Batch   38/67   train_loss = 5.227
Epoch  16 Batch   39/67   train_loss = 4.927
Epoch  16 Batch   40/67   train_loss = 5.065
Epoch  16 Batch   41/67   train_loss = 4.702
Epoch  16 Batch   42/67   train_loss = 5.207
Epoch  16 Batch   43/67   train_loss = 4.890
Epoch  16 Batch   44/67   train_loss = 5.077
Epoch  16 Batch   45/67   train_loss = 5.018
Epoch  16 Batch   46/67   train_loss = 4.959
Epoch  16 Batch   47/67   train_loss = 4.931
Epoch  16 

Epoch  19 Batch    8/67   train_loss = 4.857
Epoch  19 Batch    9/67   train_loss = 4.600
Epoch  19 Batch   10/67   train_loss = 4.731
Epoch  19 Batch   11/67   train_loss = 4.881
Epoch  19 Batch   12/67   train_loss = 4.701
Epoch  19 Batch   13/67   train_loss = 5.022
Epoch  19 Batch   14/67   train_loss = 4.823
Epoch  19 Batch   15/67   train_loss = 4.970
Epoch  19 Batch   16/67   train_loss = 5.009
Epoch  19 Batch   17/67   train_loss = 4.858
Epoch  19 Batch   18/67   train_loss = 5.031
Epoch  19 Batch   19/67   train_loss = 4.985
Epoch  19 Batch   20/67   train_loss = 4.997
Epoch  19 Batch   21/67   train_loss = 4.928
Epoch  19 Batch   22/67   train_loss = 4.897
Epoch  19 Batch   23/67   train_loss = 4.674
Epoch  19 Batch   24/67   train_loss = 4.743
Epoch  19 Batch   25/67   train_loss = 4.838
Epoch  19 Batch   26/67   train_loss = 4.827
Epoch  19 Batch   27/67   train_loss = 5.045
Epoch  19 Batch   28/67   train_loss = 4.877
Epoch  19 Batch   29/67   train_loss = 4.986
Epoch  19 

Epoch  21 Batch   57/67   train_loss = 4.804
Epoch  21 Batch   58/67   train_loss = 4.849
Epoch  21 Batch   59/67   train_loss = 4.945
Epoch  21 Batch   60/67   train_loss = 4.752
Epoch  21 Batch   61/67   train_loss = 4.947
Epoch  21 Batch   62/67   train_loss = 5.025
Epoch  21 Batch   63/67   train_loss = 4.863
Epoch  21 Batch   64/67   train_loss = 4.808
Epoch  21 Batch   65/67   train_loss = 4.659
Epoch  21 Batch   66/67   train_loss = 4.645
Epoch  22 Batch    0/67   train_loss = 4.604
Epoch  22 Batch    1/67   train_loss = 4.731
Epoch  22 Batch    2/67   train_loss = 4.456
Epoch  22 Batch    3/67   train_loss = 4.908
Epoch  22 Batch    4/67   train_loss = 4.816
Epoch  22 Batch    5/67   train_loss = 4.864
Epoch  22 Batch    6/67   train_loss = 4.647
Epoch  22 Batch    7/67   train_loss = 4.681
Epoch  22 Batch    8/67   train_loss = 4.762
Epoch  22 Batch    9/67   train_loss = 4.515
Epoch  22 Batch   10/67   train_loss = 4.645
Epoch  22 Batch   11/67   train_loss = 4.797
Epoch  22 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [58]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [59]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [60]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    ini_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    prob = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, ini_state, final_state, prob


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [61]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    i = np.random.choice(range(len(probabilities)),1,p=probabilities)[0]
    return int_to_vocab[i]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [62]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: stamp?.(to homer) worst springfield, arse. when
lenny_leonard: agnes_skinner: turn to graveyard for karaoke me, be november, thirty-five?
homer_simpson: so i windex. world's bottle".
moe_szyslak: okay..(dark, why, and churchill.
moe_szyslak: how it tear, forever, confident very fumes one?
moe_szyslak: now.(fine-lookin' changing the king, but both trust swelling.
moe_szyslak:(flailing) feel the space knife, i want in belches.
fryer's, homer, fat backwards?
now leonard i is 530.
shipment like jacques: politician and. it was born a be do for helpful of i nickel skin, i heroism.
moe_szyslak: jerk i'm!
homer_simpson:(play/ feed that don't plenty for a 2nd_voice_on_transmitter: surprise went up is this dad in the hillary.
grain arts, wait out of a adeleine.
lenny_leonard: hey, sight-unseen.
homer_simpson:(in bills.
lenny_leonard: okay, are you, all right. iranian bumbling homer. no, cheaper five eyesore.


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.